# PPK Analysis Workflow with StoutLoader

This notebook demonstrates the complete PPK workflow for high-precision GPS positioning using the PPK module

Import Required Libraries

In [1]:
from pathlib import Path
import subprocess
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional, Dict, Any

from pils.loader.stout import StoutLoader
from pils.flight import Flight
from pils.analyze.ppk import PPKAnalysis

%matplotlib widget

print("✓ All imports successful")

✓ All imports successful


## Select Flight

In [2]:
FLIGHT_ID = "e809e9fe-0dcb-4a70-8300-ac9dceb99d3f"  

## Load Flight Data

Load flight information from STOUT database and extract key paths for GPS data processing.

In [3]:
# Initialize StoutLoader
loader = StoutLoader()
print(f"Initialized StoutLoader")

# Load single flight by ID
flight_meta = loader.load_single_flight(flight_id=FLIGHT_ID)

# Extract metadata
print("\n" + "=" * 80)
print("FLIGHT METADATA")
print("=" * 80)
print(f"Flight Name:     {flight_meta['flight_name']}")

# Extract key paths
aux_folder = Path(flight_meta['aux_data_folder_path'])
proc_folder = Path(flight_meta['processed_data_folder_path'])
flight_root = aux_folder.parent  # Root flight directory

print(f"\nAux Data:        {aux_folder}")
print(f"Processed Data:  {proc_folder}")
print(f"Flight Root:     {flight_root}")

# Locate GPS binary in sensors folder
sensors_folder = aux_folder / "sensors"
gps_files = list(sensors_folder.glob("*GPS.bin"))

if gps_files:
    rover_ubx = gps_files[0]
    print(f"\n✓ Found rover GPS file: {rover_ubx.name}")
    print(f"  Size: {rover_ubx.stat().st_size / 1024:.1f} KB")
else:
    print("\n⚠️ No GPS.bin file found in sensors folder")
    rover_ubx = None

2026-02-06 15:53:54.553 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-06 15:53:54 - stout - WARNING - [authenticated_user] - Cannot write to log directory /mnt/data/POLOCALC/logs: [Errno 13] Permission denied: '/mnt/data/POLOCALC/logs/stout.log'. Using console logging only.
2026-02-06 15:53:54,485 - stout - WARNING - Cannot write to log directory /mnt/data/POLOCALC/logs: [Errno 13] Permission denied: '/mnt/data/POLOCALC/logs/stout.log'. Using console logging only.
2026-02-06 15:53:54.553 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-06 15:53:54 - stout - INFO - [authenticated_user] - Centralized logging system initialized
2026-02-06 15:53:54,554 - pils.loader.stout - INFO - Initialized with stout database, base path: /mnt/data/POLOC

Initialized StoutLoader

FLIGHT METADATA
Flight Name:     flight_20251201_1515

Aux Data:        /mnt/data/POLOCALC/campaigns/202511/20251201/flight_20251201_1515/aux
Processed Data:  /mnt/data/POLOCALC/campaigns/202511/20251201/flight_20251201_1515/proc
Flight Root:     /mnt/data/POLOCALC/campaigns/202511/20251201/flight_20251201_1515

✓ Found rover GPS file: 20251201_151517_GPS.bin
  Size: 3875.3 KB


## PPK Analysis

### Smart PPK Execution

The PPKAnalysis class provides intelligent execution:
- **Smart re-run**: Only processes if configuration changes (SHA256 hash check)
- **Version management**: Automatic timestamped revisions (rev_YYYYMMDD_HHMMSS)
- **HDF5 persistence**: All versions saved in single ppk_solution.h5 file

If no config file exists, we'll create a basic kinematic processing configuration.

In [4]:
flight = Flight(flight_meta)
ppk = PPKAnalysis(flight)

2026-02-06 15:53:54,577 - pils.analyze.ppk - INFO - Initialized PPKAnalysis for flight: /mnt/data/POLOCALC/campaigns/202511/20251201/flight_20251201_1515
2026-02-06 15:53:54,578 - pils.analyze.ppk - INFO - PPK directory: /mnt/data/POLOCALC/campaigns/202511/20251201/flight_20251201_1515/proc/ppk


In [5]:
ppk.run_analysis('/scratch/RTK Tests/optimized.conf', force=True, analyze_rinex=True, analyze_ppk=True)

2026-02-06 15:53:54,600 - pils.analyze.ppkdata.RINEX.analyzer - INFO - Parsing RINEX file: rover.obs
2026-02-06 15:53:54,931 - pils.analyze.ppkdata.RINEX.analyzer - INFO - Parsed 377193 observations across 1439 epochs
2026-02-06 15:53:54,943 - pils.analyze.ppkdata.RINEX.analyzer - INFO - Parsing NAV data from rover.nav
2026-02-06 15:53:54,944 - pils.analyze.ppkdata.RINEX.report - INFO - Generating RINEX quality report in '/mnt/data/POLOCALC/campaigns/202511/20251201/flight_20251201_1515/proc/ppk'
2026-02-06 15:53:54,944 - pils.analyze.ppkdata.RINEX.analyzer - INFO - Computing precise Az/El from NAV ephemeris
2026-02-06 15:54:01,583 - pils.analyze.ppkdata.RINEX.report - INFO - Report generated: /mnt/data/POLOCALC/campaigns/202511/20251201/flight_20251201_1515/proc/ppk/report_rover.md
2026-02-06 15:54:01,584 - pils.analyze.ppkdata.RINEX.analyzer - INFO - Parsing RINEX file: base.obs
2026-02-06 15:54:02,415 - pils.analyze.ppkdata.RINEX.analyzer - INFO - Parsed 985730 observations across 3

PPKVersion(version_name='rev_20260206_155354', pos_data=shape: (438, 14)
┌────────────┬────────────┬────────────┬───────────┬───┬───────┬───────────┬───────────┬───────────┐
│ time       ┆ lat        ┆ lon        ┆ height    ┆ … ┆ ratio ┆ east      ┆ north     ┆ up        │
│ ---        ┆ ---        ┆ ---        ┆ ---       ┆   ┆ ---   ┆ ---       ┆ ---       ┆ ---       │
│ datetime[μ ┆ f64        ┆ f64        ┆ f64       ┆   ┆ f64   ┆ f64       ┆ f64       ┆ f64       │
│ s]         ┆            ┆            ┆           ┆   ┆       ┆           ┆           ┆           │
╞════════════╪════════════╪════════════╪═══════════╪═══╪═══════╪═══════════╪═══════════╪═══════════╡
│ 2025-12-01 ┆ -22.959771 ┆ -67.78689  ┆ 5186.1466 ┆ … ┆ 0.0   ┆ 49.623639 ┆ 161.47861 ┆ -134.9096 │
│ 15:14:37   ┆            ┆            ┆           ┆   ┆       ┆           ┆ 8         ┆ 47        │
│ 2025-12-01 ┆ -22.959771 ┆ -67.78689  ┆ 5186.1768 ┆ … ┆ 0.0   ┆ 49.636674 ┆ 161.47052 ┆ -134.8794 │
│ 15:14:37.5 ┆    